<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a4bbff1ff901defaa7372a827e02b5ab57135a91fc701f9e5d825c2521679b9e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 14:08:45
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: -2.37 L (-1.71%)
Current PnL: -23.43 L (-15.76%)
CY Booked + Current PnL: -11.99 L (-8.06%)
-------------------
Total profit:  75.15 K
Total loss:  -24.18 L
-------------------
Total Booked + Current PnL: 14.99 L (12.32%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.4%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 89.29 L (65.58%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.17,35.0,M-SC,2.75,83185.0,-17592.0,17685.0,-2.79,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.81,188220.0,23550.0,19782.0,-2.34,14.30,10.51,26.31,89.0,1.19,1.38,23.43,X40N,NTT,AC
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197923.0,-2215.0,23098.0,1.16,-1.11,11.67,10.44,4.0,-0.10,1.45,4.40,X40,NTT,FMCG
50,NATIONALUM,244.55,-46.98,45.0,H-MC,10.85,105714.0,5210.0,23902.0,-2.99,5.18,22.61,28.96,79.0,0.22,0.78,40.91,MH,ATH,METALS
57,RELIANCE,1533.00,-14.41,42.0,H-LC,4.07,214360.0,3994.0,24780.0,0.12,1.90,11.56,13.68,37.0,0.16,1.57,18.74,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.0,-22.82,41.0,M-LC,2.34,244063.0,-32460.0,142533.0,1.21,-11.74,58.40,39.81,54.0,-0.23,1.79,16.97,XY24,NTT,AUTO
42,ITC,452.0,-38.77,45.0,H-LC,2.15,197923.0,-2215.0,23098.0,1.16,-1.11,11.67,10.44,4.0,-0.10,1.45,4.40,X40,NTT,FMCG
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.43,221562.0,-48105.0,84437.0,0.38,-17.84,38.11,13.47,138.0,-0.57,1.63,12.06,XY24,NTT,PAINTS
57,RELIANCE,1533.0,-14.41,42.0,H-LC,4.07,214360.0,3994.0,24780.0,0.12,1.90,11.56,13.68,37.0,0.16,1.57,18.74,XY25,NTT,REFINERIES
18,CERA,9475.0,-22.67,34.0,H-SC,1.85,140990.0,-34913.0,76938.0,0.03,-19.85,54.57,23.89,149.0,-0.45,1.04,21.76,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-5.57,28.0,H-SC,8.72,112556.0,-12741.0,71664.0,-5.08,-10.17,63.67,47.03,141.0,-0.18,0.83,40.93,MH,ATH,POWER
32,HINDZINC,730.22,19.57,50.0,M-LC,2.74,194532.0,-10544.0,123119.0,-4.19,-5.14,63.29,54.89,52.0,-0.09,1.43,17.72,X5K,ATH,METALS
60,SAIL,228.00,43.54,52.0,M-MC,11.12,226116.0,1154.0,166489.0,-3.82,0.51,73.63,74.53,192.0,0.01,1.66,32.93,XY24,BTT,STEEL
55,RAJOOENG,143.10,-44.32,22.0,H-SC,13.48,84490.0,-18010.0,58611.0,-3.81,-17.57,69.37,39.61,114.0,-0.31,0.62,0.00,AR,ATH,MISC
39,INDUSINDBK,1800.00,231.93,34.0,L-MC,12.10,40749.0,-36458.0,61849.0,-3.49,-47.22,151.78,32.89,258.0,-0.59,0.30,12.22,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,52.0,M-MC,11.12,226116.0,1154.0,166489.0,-3.82,0.51,73.63,74.53,192.0,0.01,1.66,32.93,XY24,BTT,STEEL
11,BANKINDIA,190.00,-30.43,51.0,H-MC,8.17,180100.0,292.0,112112.0,-2.22,0.16,62.25,62.52,88.0,0.00,1.32,34.30,XR,NTT,BANKS
52,PGHH,17973.08,-30.55,50.0,H-MC,3.41,201030.0,210.0,68571.0,-2.79,0.10,34.11,34.25,80.0,0.00,1.48,5.31,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,90.20,46.0,H-SC,10.13,119826.0,-22653.0,42179.0,-1.31,-15.90,35.20,13.70,163.0,-0.54,0.88,41.43,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,35.0,M-SC,2.75,83185.0,-17592.0,17685.0,-2.79,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.43,221562.0,-48105.0,84437.0,0.38,-17.84,38.11,13.47,138.0,-0.57,1.63,12.06,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,34.0,H-SC,1.85,140990.0,-34913.0,76938.0,0.03,-19.85,54.57,23.89,149.0,-0.45,1.04,21.76,OX40N,NTT,CERAMICS
66,SIS,528.0,2075.13,39.0,H-SC,4.22,87771.0,-23261.0,46870.0,-0.55,-20.95,53.40,21.26,156.0,-0.50,0.64,18.59,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,52.0,M-MC,11.12,226116.0,1154.0,166489.0,-3.82,0.51,73.63,74.53,192.0,0.01,1.66,32.93,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.81,188220.0,23550.0,19782.0,-2.34,14.30,10.51,26.31,89.0,1.19,1.38,23.43,X40N,NTT,AC
84,VOLTAS,1530.00,-3.60,40.0,H-MC,3.03,202140.0,10398.0,27370.0,-1.81,5.42,13.54,19.69,99.0,0.38,1.48,12.59,XY25,NTT,AC
17,CAMS,5211.76,-6.05,43.0,H-SC,1.77,106714.0,4710.0,39217.0,-1.48,4.62,36.75,43.06,122.0,0.12,0.78,22.52,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,50.0,M-LC,6.81,153074.0,12234.0,32253.0,-2.98,8.69,21.07,31.59,66.0,0.38,1.12,37.48,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.98,45.0,H-MC,10.85,105714.0,5210.0,23902.0,-2.99,5.18,22.61,28.96,79.0,0.22,0.78,40.91,MH,ATH,METALS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.24,125318.0,1982.0,131784.0,-0.38,1.61,105.16,108.45,119.0,0.02,0.92,24.32,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-45.21,30.0,M-SC,1.51,92535.0,1036.0,77711.0,-1.22,1.13,83.98,86.07,223.0,0.01,0.68,34.38,XR,NTT,DURABLES
11,BANKINDIA,190.00,-30.43,51.0,H-MC,8.17,180100.0,292.0,112112.0,-2.22,0.16,62.25,62.52,88.0,0.00,1.32,34.30,XR,NTT,BANKS
86,WIPRO,420.00,-16.97,32.0,M-LC,5.65,145855.0,-5090.0,114540.0,-2.79,-3.37,78.53,72.51,53.0,-0.04,1.07,2.45,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.50,27.0,H-LC,12.94,254130.0,-13235.0,78679.0,-2.02,-4.95,30.96,24.48,48.0,-0.17,1.87,47.30,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-18.03,28.0,H-LC,7.02,206985.0,-44783.0,97987.0,-2.16,-17.79,47.34,21.13,27.0,-0.46,1.52,11.34,X40,ATH,PAINTS
74,TCS,4476.75,-31.90,28.0,H-LC,12.08,271933.0,-74023.0,148883.0,-2.18,-21.40,54.75,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.95,30.0,H-LC,8.21,284739.0,-31103.0,143765.0,-1.23,-9.85,50.49,35.67,5.0,-0.22,2.09,2.64,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,9.96,218670.0,-23254.0,86528.0,-2.18,-9.61,39.57,26.15,8.0,-0.27,1.61,3.33,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,40.0,H-LC,1.67,229275.0,-16876.0,96089.0,-2.28,-6.86,41.91,32.18,16.0,-0.18,1.68,26.72,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,2.15,197923.0,-2215.0,23098.0,1.16,-1.11,11.67,10.44,4.0,-0.10,1.45,4.40,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.90,40.0,H-LC,2.64,154490.0,-31605.0,79083.0,-2.11,-16.98,51.19,25.51,15.0,-0.40,1.13,13.85,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,42.0,H-LC,4.07,214360.0,3994.0,24780.0,0.12,1.90,11.56,13.68,37.0,0.16,1.57,18.74,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.41,43.0,H-LC,4.34,272540.0,7114.0,49684.0,-0.75,2.68,18.23,21.40,11.0,0.14,2.00,10.21,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.90,28.0,H-LC,12.08,271933.0,-74023.0,148883.0,-2.18,-21.40,54.75,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.66,43.0,H-LC,7.17,223686.0,-10458.0,59075.0,-1.30,-4.47,26.41,20.77,86.0,-0.18,1.64,2.14,X40N,NTT,BREWERIES
83,VBL,671.64,-20.95,30.0,H-LC,8.21,284739.0,-31103.0,143765.0,-1.23,-9.85,50.49,35.67,5.0,-0.22,2.09,2.64,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,9.96,218670.0,-23254.0,86528.0,-2.18,-9.61,39.57,26.15,8.0,-0.27,1.61,3.33,X40,ATH,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197923.0,-2215.0,23098.0,1.16,-1.11,11.67,10.44,4.0,-0.10,1.45,4.40,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.90,40.0,H-LC,2.64,154490.0,-31605.0,79083.0,-2.11,-16.98,51.19,25.51,15.0,-0.40,1.13,13.85,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197923.0,-2215.0,23098.0,1.16,-1.11,11.67,10.44,4.0,-0.10,1.45,4.40,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.54,204815.0,315.0,41106.0,-0.97,0.15,20.07,20.25,10.0,0.01,1.50,8.90,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.03,28.0,H-LC,7.02,206985.0,-44783.0,97987.0,-2.16,-17.79,47.34,21.13,27.0,-0.46,1.52,11.34,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,42.0,H-LC,4.07,214360.0,3994.0,24780.0,0.12,1.90,11.56,13.68,37.0,0.16,1.57,18.74,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.90,28.0,H-LC,12.08,271933.0,-74023.0,148883.0,-2.18,-21.40,54.75,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.14,308104.0,-4784.0,176482.0,-2.58,-1.53,57.28,54.87,3.0,-0.03,2.26,5.02,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197923.0,-2215.0,23098.0,1.16,-1.11,11.67,10.44,4.0,-0.10,1.45,4.40,X40,NTT,FMCG
83,VBL,671.64,-20.95,30.0,H-LC,8.21,284739.0,-31103.0,143765.0,-1.23,-9.85,50.49,35.67,5.0,-0.22,2.09,2.64,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,42.0,H-LC,7.98,247253.0,-14366.0,133591.0,-0.95,-5.49,54.03,45.57,7.0,-0.11,1.82,5.21,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,41.33,78720.0,-34829.0,74878.0,-3.23,-30.67,95.12,35.27,268.0,-0.47,0.58,92.67,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,39.0,H-SC,12.49,91230.0,-6750.0,28354.0,-1.52,-6.89,31.08,22.05,152.0,-0.24,0.67,32.61,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.98,45.0,H-MC,10.85,105714.0,5210.0,23902.0,-2.99,5.18,22.61,28.96,79.0,0.22,0.78,40.91,MH,ATH,METALS
78,UJJIVANSFB,60.00,90.20,46.0,H-SC,10.13,119826.0,-22653.0,42179.0,-1.31,-15.90,35.20,13.70,163.0,-0.54,0.88,41.43,OX40N,NTT,BANKS
79,UNIONBANK,163.00,-14.73,50.0,M-LC,6.81,153074.0,12234.0,32253.0,-2.98,8.69,21.07,31.59,66.0,0.38,1.12,37.48,XY24,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1153.59,61.0,M-SC,12.61,170485.0,-10658.0,56124.0,-1.98,-5.88,32.92,25.10,235.0,-0.19,1.25,36.63,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-170.45,54.0,M-SC,5.85,81600.0,-20610.0,114003.0,-0.30,-20.16,139.71,91.37,208.0,-0.18,0.60,32.46,XY25,NTT,FINANCE
7,ATULAUTO,844.00,3554.05,48.0,M-SC,7.09,113964.0,-33404.0,85222.0,-1.79,-22.67,74.78,35.16,236.0,-0.39,0.84,16.40,XY24,NTT,AUTO
71,TANLA,1963.11,-20.33,54.0,H-SC,16.94,213649.0,-57112.0,387068.0,-3.12,-21.09,181.17,121.86,133.0,-0.15,1.57,67.13,AR,ATH,IT
81,VAIBHAVGBL,521.00,51.52,48.0,H-SC,5.65,131587.0,-51188.0,162260.0,-3.20,-28.01,123.31,60.77,125.0,-0.32,0.97,17.86,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.30
1,25,43.82
2,50,74.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.84
LC    32.45
MC    23.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.35
X40      14.76
X40N     11.37
XR       11.13
XY25     10.19
AR        8.30
OX40N     7.92
X200      1.68
MH        1.61
X5K       1.43
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.34
H-LC    25.98
H-MC    20.73
M-SC    13.03
M-LC     5.41
M-MC     2.64
L-SC     1.47
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.10,-6.81,42.06
IT,12.83,-20.41,86.06
FINANCE,10.04,-20.01,72.37
MISC,6.98,-20.77,87.91
BANKS,6.14,-19.35,82.13
PAINTS,5.71,-18.90,36.62
ELECTRICAL,5.48,-10.40,49.14
INSURANCE,3.80,-6.92,46.88
AC,3.57,0.62,16.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3247826.0,22
XR,1397049.0,14
AR,1200336.0,9
X40,839652.0,10
X40N,654215.0,9
OX40N,569511.0,10
XY25,450032.0,7
SR,255937.0,2
X5K,123119.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3799880.0,29
M-SC,1470708.0,17
H-MC,1284094.0,15
H-LC,1279957.0,15
M-LC,412445.0,4
M-MC,318188.0,2
L-SC,262066.0,3
L-MC,61849.0,1
L-LC,40145.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257317.0      6
M-SC       XY24       842456.0      7
H-SC       AR         816495.0      5
           XR         809340.0      7
H-MC       XY24       576400.0      4
H-LC       X40        532978.0      5
H-SC       X40N       321860.0      4
M-MC       XY24       318188.0      2
H-SC       OX40N      267267.0      4
           SR         255937.0      2
H-LC       X40N       243946.0      3
H-MC       X40        229982.0      4
H-LC       AR         212674.0      2
H-MC       XY25       180293.0      2
M-LC       XY24       174786.0      2
L-SC       XR         173156.0      2
M-SC       AR         171167.0      2
           OX40N      140338.0      4
           XR         126052.0      2
M-LC       X5K        123119.0      1
H-LC       XY25       115591.0      3
M-LC       XR         114540.0      1
M-SC       XY25       114003.0      1
H-MC       XR         112112.0      1
H-LC       X200        96089.0      1
L-SC       OX40N       88910.0      1
H-MC       X40N        88409.0      2
H-LC       XY24        78679.0      1
M-SC       X40         76692.0      1
H-MC       OX40N       72996.0      1
H-SC       MH          71664.0      1
L-MC       XR          61849.0      1
L-LC       XY25        40145.0      1
H-MC       MH          23902.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
